In [2]:
# hyperopt==0.2.5
import hyperopt
import pandas as pd
import numpy as np
from hpsklearn import HyperoptEstimator,any_regressor, random_forest_regression, any_classifier, any_preprocessing
from hpsklearn import one_hot_encoder, pca, standard_scaler, min_max_scaler, normalizer, ts_lagselector, tfidf, rbm, colkmeans
from sklearn.datasets import load_iris
from hyperopt import tpe
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
import numpy as np
from PIL import Image
import os as os

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


# Overview of data, split train and test

In [12]:
data_dir = "data"
val_df = pd.read_csv(f'{data_dir}/test.csv')
train_df = pd.read_csv(f'{data_dir}/train.csv')

display(train_df.head(3))
display(val_df.head(3))

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1


In [42]:
train_df.apply(lambda x: x.unique()).head()

Id               [0007de18844b0dbbb5e1f607da0606e0, 0009c66b943...
Subject Focus                                               [0, 1]
Eyes                                                        [1, 0]
Face                                                        [1, 0]
Near                                                        [1, 0]
dtype: object

In [16]:
X = train_df.iloc[:, :(train_df.shape[1] -1)].drop("Id", axis = 1)
Y = train_df.iloc[:, train_df.shape[1] - 1]
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y,
                                                    random_state = 0,
                                                   train_size= 0.2)

sample_submission.csv  test  test.csv  train  train.csv


# Hyperopt

## As a classification task

In [26]:
estim = HyperoptEstimator(classifier= any_classifier('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=10,
                          trial_timeout=120)

estim.fit(X_train, Y_train)

print(f'Estimated score: {estim.score(X_test, Y_test)}')
print(f'Estimated best model: {estim.best_model()}')

 67%|██████▋   | 2/3 [00:00<?, ?trial/s, best loss=?]

/home/revilo/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[14:07:33] WARNING: ../src/learner.cc:541:           
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:07:33] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 75%|███████▌  | 3/4 [00:00<?, ?trial/s, best loss=?]

/home/revilo/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[14:08:15] WARNING: ../src/learner.cc:541:           
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:08:15] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
100%|██████████| 10/10 [00:00<00:00,  7.52trial/s, best loss: 0.9672544080604534]
0.03114754098360656
{'learner': RandomForestClassifier(bootstrap=False, max_depth=2, max_features='sqrt',
                       min_samples_leaf=2, n_estimators=807, n_jobs=1,
                       random_state=3, verbose=False), 'preprocs': (), 'ex_preprocs': ()}


## As a regression task

### Any preprocessing + random_forest_regression

In [27]:
estim = HyperoptEstimator(regressor= random_forest_regression('my_rf_reg'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

estim.fit(X_train, Y_train)

print(estim.score(X_test, Y_test))
print(estim.best_model())

100%|██████████| 100/100 [00:00<00:00,  7.27trial/s, best loss: 0.9977569924934134]
-0.005490063841474546
{'learner': RandomForestRegressor(max_depth=3, max_features=0.891101001609195, n_jobs=1,
                      random_state=2, verbose=False), 'preprocs': (Normalizer(),), 'ex_preprocs': ()}


### Selected preprocessing

In [38]:
estim = HyperoptEstimator(regressor= random_forest_regression('my_rf_reg'),
                          preprocessing= [pca('my_pre')],
                          algo=tpe.suggest,
                          max_evals=50,
                          trial_timeout=120)

estim.fit(X_train, Y_train)

print(f'Estimated score: {estim.score(X_test, Y_test)}')
print(f'Estimated best model: {estim.best_model()}')

100%|██████████| 100/100 [00:00<00:00,  7.06trial/s, best loss: 0.9958682465628795]
Estimated score: -0.004382587793090531
Estimated best model: {'learner': RandomForestRegressor(max_depth=3, max_features='log2', n_estimators=94,
                      n_jobs=1, random_state=4, verbose=False), 'preprocs': (PCA(n_components=12),), 'ex_preprocs': ()}


# Hyperopt with sklearn.MLP

https://github.com/bjkomer/hyperopt-tutorial/blob/master/MNIST-Example.ipynb

In [22]:
#selected_no_of_images = 50
training_images_dir = "data/train_shrinked/"
training_fnames = os.listdir(training_images_dir)

pixel_array_list = []

lens = []
for fname in training_fnames:
    im = Image.open(f'{training_images_dir}{fname}')
    length = np.array(list(im.getdata())).shape[0]
    lens.append(length)

min_len = np.min(lens)


In [23]:
min_len

5248

In [24]:
for fname in training_fnames:
    im = Image.open(f'{training_images_dir}{fname}')
    
    pix = im.load()
    im_array = np.array(list(im.getdata()))
    im_array_gray = np.mean(im_array, axis = 1)[:(min_len - 1)] # cut to minimum
    pixel_array_list.append(im_array_gray)

#pixels_df = pd.concat(pixel_array_list, axis = 0)
    

In [25]:
np.vstack(pixel_array_list).shape

(9912, 5247)

In [26]:
data_dir = "data"
train_df = pd.read_csv(f'{data_dir}/train.csv')
#val_df = pd.read_csv(f'{data_dir}/test.csv')

In [46]:
X = np.vstack(pixel_array_list) / 255.
matches = [np.where(x == train_df.Id.values)[0][0] for x in [y[:len(y)-4] for y in os.listdir("data/train_shrinked/")]]
y = train_df.iloc[matches,:].Pawpularity.to_numpy()

In [47]:
# Split into training and testing sets
X_train, X_test = X[:7000], X[7000:]
y_train, y_test = y[:7000], y[7000:]

# Define an objective function to minimize
# The classifier will be created, trained, and scored within this function
def objective(args):
    
    # Build a classifier based on the parameters chosen
    mlp = MLPClassifier(hidden_layer_sizes=(int(args['layer_size']),), max_iter=10,
                        alpha=args['alpha'], solver=args['algorithm'], tol=1e-4, 
                        random_state=1, activation=args['activation'], 
                        learning_rate_init=args['learning_rate']
                       )
    
    # Fit the classifier to the training data
    mlp.fit(X_train, y_train)
    
    #NOTE: Normally you should use a separate 'validation' set here
    #      and have a 'test' set that is only used on the final classifier
    #      once parameters have been selected, the final classifier can be
    #      retrained on both the 'training' and 'validation' sets
    loss = -mlp.score(X_test, y_test)
    
    # Must return loss and status, any additional information can also be saved here.
    # In this example the fully trained model is also returned
    return {'loss': loss, 'status': STATUS_OK, 'model':mlp}

# Define the parameter space to search over
# In this case the objective function is expecting a single dictionary argument, 
# so the space variable is set up to match that
space = {'layer_size':hp.quniform('layer_size', 25, 100, 1),
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'algorithm':hp.choice('algorithm', ['lbfgs', 'sgd', 'adam']),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),
         #'learning_rate':hp.uniform('learning_rate', low=0.001, high=0.999),
         'learning_rate':hp.loguniform('learning_rate', low=np.log(1e-4), high=np.log(1.)),
        }

# Create a Trials object to store results of each evaluation
trials = Trials()

# Run the search for the specified number of evaluations
best = fmin(objective,
            space=space,
            algo=tpe.suggest,
            trials=trials,
            max_evals=20)

# Get the trained model from the best trial
best_model = trials.best_trial['result']['model']

# Compute the training and testing scores on this model
print("Training Accuracy: %f" % best_model.score(X_train, y_train))
print("Testing Accuracy: %f" % best_model.score(X_test, y_test))

print(best)
print(trials.best_trial)



  5%|▌         | 1/20 [00:03<01:14,  3.90s/trial, best loss: -0.029876373626373628]

/home/revilo/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(



 10%|█         | 2/20 [00:06<01:02,  3.45s/trial, best loss: -0.029876373626373628]

/home/revilo/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(



 15%|█▌        | 3/20 [00:07<00:49,  2.91s/trial, best loss: -0.03159340659340659] 

...

 95%|█████████▌| 19/20 [00:50<00:02,  2.53s/trial, best loss: -0.034340659340659344]

/home/revilo/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(



100%|██████████| 20/20 [00:52<00:00,  2.64s/trial, best loss: -0.034340659340659344]

/home/revilo/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)




Training Accuracy: 0.033000
Testing Accuracy: 0.034341
{'activation': 0, 'algorithm': 1, 'alpha': 0.0001336183842118413, 'layer_size': 93.0, 'learning_rate': 0.6029980867483039}
{'state': 2, 'tid': 5, 'spec': None, 'result': {'loss': -0.034340659340659344, 'status': 'ok', 'model': MLPClassifier(activation='logistic', alpha=0.0001336183842118413,
              hidden_layer_sizes=(93,), learning_rate_init=0.6029980867483039,
              max_iter=10, random_state=1, solver='sgd')}, 'misc': {'tid': 5, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'activation': [5], 'algorithm': [5], 'alpha': [5], 'layer_size': [5], 'learning_rate': [5]}, 'vals': {'activation': [0], 'algorithm': [1], 'alpha': [0.0001336183842118413], 'layer_size': [93.0], 'learning_rate': [0.6029980867483039]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2021, 12, 29, 19, 10, 12, 14000), 'refresh_time': datetime.datetime(2021, 12, 29, 19, 10, 15, 84000)}


## Mnist example

In [56]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
import numpy as np

#mnist = fetch_mldata('MNIST original')
mnist = fetch_openml('mnist_784')

# Scale the images to be between 0 and 1
X = mnist.data / 255.
y = mnist.target

# Split into training and testing sets
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

# Define an objective function to minimize
# The classifier will be created, trained, and scored within this function
def objective(args):
    
    # Build a classifier based on the parameters chosen
    mlp = MLPClassifier(hidden_layer_sizes=(int(args['layer_size']),), max_iter=10,
                        alpha=args['alpha'], solver=args['algorithm'], tol=1e-4, 
                        random_state=1, activation=args['activation'], 
                        learning_rate_init=args['learning_rate']
                       )
    
    # Fit the classifier to the training data
    mlp.fit(X_train, y_train)
    
    #NOTE: Normally you should use a separate 'validation' set here
    #      and have a 'test' set that is only used on the final classifier
    #      once parameters have been selected, the final classifier can be
    #      retrained on both the 'training' and 'validation' sets
    loss = -mlp.score(X_test, y_test)
    
    # Must return loss and status, any additional information can also be saved here.
    # In this example the fully trained model is also returned
    return {'loss': loss, 'status': STATUS_OK, 'model':mlp}

# Define the parameter space to search over
# In this case the objective function is expecting a single dictionary argument, 
# so the space variable is set up to match that
space = {'layer_size':hp.quniform('layer_size', 25, 100, 1),
         'alpha':hp.lognormal('alpha', mu=np.log(1e-4), sigma=1),
         'algorithm':hp.choice('algorithm', ['lbfgs', 'sgd', 'adam']),
         'activation':hp.choice('activation', ['logistic', 'tanh', 'relu']),
         #'learning_rate':hp.uniform('learning_rate', low=0.001, high=0.999),
         'learning_rate':hp.loguniform('learning_rate', low=np.log(1e-4), high=np.log(1.)),
        }

# Create a Trials object to store results of each evaluation
trials = Trials()

# Run the search for the specified number of evaluations
best = fmin(objective,
            space=space,
            algo=tpe.suggest,
            trials=trials,
            max_evals=10)

# Get the trained model from the best trial
best_model = trials.best_trial['result']['model']

# Compute the training and testing scores on this model
print("Training Accuracy: %f" % best_model.score(X_train, y_train))
print("Testing Accuracy: %f" % best_model.score(X_test, y_test))

print(best)
print(trials.best_trial)




 10%|█         | 1/10 [00:05<00:50,  5.61s/trial, best loss: -0.9524]

/home/revilo/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(



 20%|██        | 2/10 [00:08<00:38,  4.84s/trial, best loss: -0.9524]

/home/revilo/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)



 30%|███       | 3/10 [00:15<00:37,  5.34s/trial, best loss: -0.9524]

...

 90%|█████████ | 9/10 [00:57<00:06,  6.69s/trial, best loss: -0.9703]

/home/revilo/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(



100%|██████████| 10/10 [01:04<00:00,  6.43s/trial, best loss: -0.9703]

/home/revilo/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(




Training Accuracy: 0.981250
Testing Accuracy: 0.970300
{'activation': 2, 'algorithm': 1, 'alpha': 0.0002032864657772866, 'layer_size': 84.0, 'learning_rate': 0.024929603536400023}
{'state': 2, 'tid': 4, 'spec': None, 'result': {'loss': -0.9703, 'status': 'ok', 'model': MLPClassifier(alpha=0.0002032864657772866, hidden_layer_sizes=(84,),
              learning_rate_init=0.024929603536400023, max_iter=10,
              random_state=1, solver='sgd')}, 'misc': {'tid': 4, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'activation': [4], 'algorithm': [4], 'alpha': [4], 'layer_size': [4], 'learning_rate': [4]}, 'vals': {'activation': [2], 'algorithm': [1], 'alpha': [0.0002032864657772866], 'layer_size': [84.0], 'learning_rate': [0.024929603536400023]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2021, 12, 29, 17, 55, 41, 835000), 'refresh_time': datetime.datetime(2021, 12, 29, 17, 55, 49, 866000)}


In [57]:
trials.results

[{'loss': -0.9524,
  'status': 'ok',
  'model': MLPClassifier(activation='logistic', alpha=0.0005181376624604576,
                hidden_layer_sizes=(39,), learning_rate_init=0.04838476009377832,
                max_iter=10, random_state=1, solver='sgd')},
 {'loss': -0.8715,
  'status': 'ok',
  'model': MLPClassifier(alpha=3.6647592130329364e-05, hidden_layer_sizes=(70,),
                learning_rate_init=0.01895107026278117, max_iter=10,
                random_state=1, solver='lbfgs')},
 {'loss': -0.9245,
  'status': 'ok',
  'model': MLPClassifier(activation='tanh', alpha=0.00012916337960198752,
                hidden_layer_sizes=(46,),
                learning_rate_init=0.0027405589150218323, max_iter=10,
                random_state=1, solver='sgd')},
 {'loss': -0.0894,
  'status': 'ok',
  'model': MLPClassifier(alpha=0.0002968708755733914, hidden_layer_sizes=(40,),
                learning_rate_init=0.5928035423594112, max_iter=10,
                random_state=1)},
 {'loss': -0.97